In [1]:
###### Importing Libraries
import os
import torch
import torchtext
import nltk
import spacy
import numpy as np
import pandas as pd
from torchtext import data
from sklearn.model_selection import train_test_split

c:\users\asus\anaconda3\envs\gpu_torch_3\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
###### Data Split
csv_op = pd.read_csv('./Reviews_q2_main.csv',index_col=False)
csv_train, csv_test = train_test_split(csv_op,train_size=0.8,random_state=244)
csv_val, csv_test = train_test_split(csv_test,train_size=0.75,random_state=244)

######  Writing CSV files
csv_train.to_csv('./train.csv')
csv_val.to_csv('./val.csv')
csv_test.to_csv('./test.csv')

In [6]:
vocab,embeddings = [],[]
with open('./glove.twitter.27B/glove.twitter.27B.100d.txt','rt') as fi:
    full_content = fi.read().strip().split('\n')
for i in range(len(full_content)):
    i_word = full_content[i].split(' ')[0]
    #i_embeddings = [float(val) for val in full_content[i].split(' ')[1:]]
    vocab.append(i_word)
    #embeddings.append(i_embeddings)

vocab_npa = np.array(vocab)
embs_npa = np.array(embeddings)

vocab_npa = np.insert(vocab_npa, 0, '<pad>')
vocab_npa = np.insert(vocab_npa, 1, '<unk>')
print(vocab_npa[:10])

['<pad>' '<unk>' '<user>' '.' ':' 'rt' ',' '<repeat>' '<hashtag>'
 '<number>']


In [7]:
def tokens2indices(tokens,pad_len):

    word2idx = {term:idx for idx,term in enumerate(vocab_npa)}
    idx2word = {idx:word for word,idx in word2idx.items()}
    
    ##### Padding
    missing_len = abs(pad_len - len(tokens))

    if(len(tokens) >= pad_len):
        tokens = tokens[:pad_len]
    else:
        for _ in range(pad_len - len(tokens)):
            tokens.append('<pad>')

    ##### Index conversion
    for idx in range(len(tokens)):

        if tokens[idx] not in word2idx:
            tokens[idx] = word2idx['<unk>']

        else:
            tokens[idx] = word2idx[tokens[idx]]

    return torch.Tensor(tokens).long(), missing_len

In [8]:
tweet = '@aminbaybon HEllllllllo everybody i wish you a good #day'
tokenizer = nltk.tokenize.TweetTokenizer()
tweet_tokens = tokenizer.tokenize(tweet)
print(tweet_tokens)

['@aminbaybon', 'HEllllllllo', 'everybody', 'i', 'wish', 'you', 'a', 'good', '#day']


In [11]:
a,b = tokens2indices(tweet_tokens,50)

torch.Size([50])
